In [ ]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.dates as mdates
import matplotlib.patches as patches
import talib
from typing import Tuple, Optional, List
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class MACDTradingStrategy:
    """
    A comprehensive MACD trading strategy implementation with improved features.
    """

    def __init__(self, fast_period: int = 12, slow_period: int = 26, signal_period: int = 9):
        """
        Initialize MACD parameters.

        Args:
            fast_period: Fast EMA period (default: 12)
            slow_period: Slow EMA period (default: 26)
            signal_period: Signal line EMA period (default: 9)
        """
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.signal_period = signal_period

    def load_and_prepare_data(self, csv_file: str) -> pd.DataFrame:
        """
        Load MT5 CSV data and prepare it for analysis.

        Args:
            csv_file: Path to the CSV file

        Returns:
            Prepared DataFrame with DateTime index
        """
        try:
            df = pd.read_csv(csv_file)
            df['DateTime'] = pd.to_datetime(df['DateTime'])
            df.set_index('DateTime', inplace=True)

            # Ensure required columns exist
            required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
            missing_cols = [col for col in required_cols if col not in df.columns]
            if missing_cols:
                raise ValueError(f"Missing required columns: {missing_cols}")

            return df[required_cols].copy()

        except Exception as e:
            print(f"Error loading data: {e}")
            raise

    def calculate_macd(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate MACD indicators and add them to the DataFrame.

        Args:
            df: Input DataFrame with OHLC data

        Returns:
            DataFrame with MACD indicators added
        """
        df = df.copy()

        # Calculate MACD using talib
        macd_line, signal_line, macd_hist = talib.MACD(
            df['Close'],
            fastperiod=self.fast_period,
            slowperiod=self.slow_period,
            signalperiod=self.signal_period
        )

        df['MACD'] = macd_line
        df['MACD_Signal'] = signal_line
        df['MACD_Hist'] = macd_hist

        return df

    def detect_macd_crossovers(self, df: pd.DataFrame) -> pd.Series:
        """
        Detect MACD crossovers with improved logic.

        Args:
            df: DataFrame with MACD and Signal columns

        Returns:
            Series with crossover signals: 1 (bullish), -1 (bearish), 0 (none)
        """
        crossovers = pd.Series(0, index=df.index, name='Crossover')

        # Ensure we have valid MACD data
        valid_mask = ~(df['MACD'].isna() | df['MACD_Signal'].isna())

        if valid_mask.sum() < 2:  # Need at least 2 valid points
            return crossovers

        # Bullish crossover: MACD crosses above Signal
        bullish_mask = (
            (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)) &
            (df['MACD'] > df['MACD_Signal']) &
            valid_mask
        )
        crossovers[bullish_mask] = 1

        # Bearish crossover: MACD crosses below Signal
        bearish_mask = (
            (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)) &
            (df['MACD'] < df['MACD_Signal']) &
            valid_mask
        )
        crossovers[bearish_mask] = -1

        return crossovers

    def calculate_stop_loss_take_profit(self, df: pd.DataFrame,
                                      sl_lookback: int = 1,
                                      tp_pips: float = 5.0,
                                      atr_period: int = 14,
                                      use_atr: bool = True) -> pd.DataFrame:
        """
        Calculate stop loss and take profit levels with improved logic.

        Args:
            df: DataFrame with OHLC and crossover data
            sl_lookback: Number of candles to look back for SL calculation
            tp_pips: Take profit in pips (or price units)
            atr_period: Period for ATR calculation
            use_atr: Whether to use ATR-based SL/TP

        Returns:
            DataFrame with SL and TP columns added
        """
        df = df.copy()

        # Initialize SL and TP columns
        df['SL'] = np.nan
        df['TP'] = np.nan

        if use_atr:
            # Calculate ATR for dynamic SL/TP
            df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=atr_period)

        # Bullish crossovers
        bullish_mask = df['Crossover'] == 1
        if bullish_mask.any():
            if use_atr and 'ATR' in df.columns:
                # ATR-based SL: 1.5 * ATR below entry
                df.loc[bullish_mask, 'SL'] = df['Close'] - (1.5 * df['ATR'])
                # ATR-based TP: 2 * ATR above entry
                df.loc[bullish_mask, 'TP'] = df['Close'] + (2 * df['ATR'])
            else:
                # Fixed pip-based approach
                df.loc[bullish_mask, 'SL'] = df['Low'].shift(sl_lookback)
                df.loc[bullish_mask, 'TP'] = df['Close'] + tp_pips

        # Bearish crossovers
        bearish_mask = df['Crossover'] == -1
        if bearish_mask.any():
            if use_atr and 'ATR' in df.columns:
                df.loc[bearish_mask, 'SL'] = df['Close'] + (1.5 * df['ATR'])
                df.loc[bearish_mask, 'TP'] = df['Close'] - (2 * df['ATR'])
            else:
                df.loc[bearish_mask, 'SL'] = df['High'].shift(sl_lookback)
                df.loc[bearish_mask, 'TP'] = df['Close'] - tp_pips

        return df

    def add_additional_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Add additional technical indicators for enhanced analysis.

        Args:
            df: Input DataFrame

        Returns:
            DataFrame with additional indicators
        """
        df = df.copy()

        # Moving averages
        df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)
        df['EMA_50'] = talib.EMA(df['Close'], timeperiod=50)
        df['SMA_200'] = talib.SMA(df['Close'], timeperiod=200)

        # RSI
        df['RSI'] = talib.RSI(df['Close'], timeperiod=14)

        # Bollinger Bands
        df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = talib.BBANDS(
            df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2
        )

        return df

    def filter_signals(self, df: pd.DataFrame,
                      use_trend_filter: bool = True,
                      use_rsi_filter: bool = True,
                      rsi_oversold: float = 30,
                      rsi_overbought: float = 70) -> pd.DataFrame:
        """
        Apply filters to improve signal quality.

        Args:
            df: DataFrame with signals and indicators
            use_trend_filter: Filter signals based on trend direction
            use_rsi_filter: Filter signals based on RSI levels
            rsi_oversold: RSI oversold threshold
            rsi_overbought: RSI overbought threshold

        Returns:
            DataFrame with filtered signals
        """
        df = df.copy()
        df['Filtered_Crossover'] = df['Crossover'].copy()

        if use_trend_filter and all(col in df.columns for col in ['EMA_20', 'EMA_50']):
            # Only take bullish signals when price is above EMA_50 (uptrend)
            uptrend_mask = df['Close'] > df['EMA_50']
            df.loc[(df['Crossover'] == 1) & (~uptrend_mask), 'Filtered_Crossover'] = 0

            # Only take bearish signals when price is below EMA_50 (downtrend)
            downtrend_mask = df['Close'] < df['EMA_50']
            df.loc[(df['Crossover'] == -1) & (~downtrend_mask), 'Filtered_Crossover'] = 0

        if use_rsi_filter and 'RSI' in df.columns:
            # Avoid bullish signals when RSI is overbought
            df.loc[(df['Crossover'] == 1) & (df['RSI'] > rsi_overbought), 'Filtered_Crossover'] = 0

            # Avoid bearish signals when RSI is oversold
            df.loc[(df['Crossover'] == -1) & (df['RSI'] < rsi_oversold), 'Filtered_Crossover'] = 0

        return df

    def backtest_strategy(self, df: pd.DataFrame,
                         initial_balance: float = 10000,
                         risk_per_trade: float = 0.02) -> dict:
        """
        Simple backtest of the strategy.

        Args:
            df: DataFrame with signals and SL/TP levels
            initial_balance: Starting account balance
            risk_per_trade: Risk percentage per trade

        Returns:
            Dictionary with backtest results
        """
        balance = initial_balance
        trades = []
        in_trade = False
        entry_price = 0
        trade_type = 0
        sl_price = 0
        tp_price = 0

        for idx, row in df.iterrows():
            if not in_trade and row['Filtered_Crossover'] != 0:
                # Enter trade
                in_trade = True
                entry_price = row['Close']
                trade_type = row['Filtered_Crossover']
                sl_price = row['SL']
                tp_price = row['TP']

            elif in_trade:
                # Check exit conditions
                hit_sl = False
                hit_tp = False

                if trade_type == 1:  # Long position
                    hit_sl = row['Low'] <= sl_price
                    hit_tp = row['High'] >= tp_price
                else:  # Short position
                    hit_sl = row['High'] >= sl_price
                    hit_tp = row['Low'] <= tp_price

                if hit_sl or hit_tp:
                    exit_price = sl_price if hit_sl else tp_price

                    # Calculate P&L
                    if trade_type == 1:
                        pnl = exit_price - entry_price
                    else:
                        pnl = entry_price - exit_price

                    # Calculate position size based on risk
                    risk_amount = balance * risk_per_trade
                    if abs(entry_price - sl_price) > 0:
                        position_size = risk_amount / abs(entry_price - sl_price)
                    else:
                        position_size = 0

                    trade_pnl = pnl * position_size
                    balance += trade_pnl

                    trades.append({
                        'entry_date': idx,
                        'entry_price': entry_price,
                        'exit_price': exit_price,
                        'type': 'Long' if trade_type == 1 else 'Short',
                        'pnl': trade_pnl,
                        'hit_sl': hit_sl,
                        'balance': balance
                    })

                    in_trade = False

        # Calculate statistics
        if trades:
            total_trades = len(trades)
            winning_trades = sum(1 for t in trades if t['pnl'] > 0)
            win_rate = winning_trades / total_trades * 100
            total_pnl = sum(t['pnl'] for t in trades)
            max_drawdown = 0
            peak = initial_balance

            for trade in trades:
                if trade['balance'] > peak:
                    peak = trade['balance']
                drawdown = (peak - trade['balance']) / peak * 100
                max_drawdown = max(max_drawdown, drawdown)
        else:
            total_trades = win_rate = total_pnl = max_drawdown = 0

        return {
            'total_trades': total_trades,
            'win_rate': win_rate,
            'total_pnl': total_pnl,
            'final_balance': balance,
            'max_drawdown': max_drawdown,
            'trades': trades
        }

    def plot_enhanced_chart(self, df: pd.DataFrame,
                          start_idx: Optional[int] = None,
                          end_idx: Optional[int] = None,
                          show_signals: bool = True,
                          show_boxes: bool = True) -> Tuple:
        """
        Create an enhanced chart with multiple panels and indicators.

        Args:
            df: DataFrame with all indicators
            start_idx: Start index for plotting
            end_idx: End index for plotting
            show_signals: Whether to show crossover signals
            show_boxes: Whether to show SL/TP boxes

        Returns:
            Tuple of (figure, axes)
        """
        # Slice data if indices provided
        if start_idx is not None:
            df = df.iloc[start_idx:end_idx] if end_idx else df.iloc[start_idx:]

        df = df.copy().reset_index()

        # Prepare additional plots
        addplots = []

        # Moving averages on main panel
        if 'EMA_20' in df.columns:
            addplots.append(mpf.make_addplot(df['EMA_20'], color='orange', width=1))
        if 'EMA_50' in df.columns:
            addplots.append(mpf.make_addplot(df['EMA_50'], color='blue', width=1))
        if 'SMA_200' in df.columns:
            addplots.append(mpf.make_addplot(df['SMA_200'], color='red', width=1))

        # MACD on panel 1
        if 'MACD' in df.columns:
            addplots.append(mpf.make_addplot(df['MACD'], panel=1, color='green', width=1.2))
            addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=1, color='red', width=1.2))
            addplots.append(mpf.make_addplot(df['MACD_Hist'], panel=1, type='bar', color='gray', alpha=0.5))

        # RSI on panel 2
        if 'RSI' in df.columns:
            addplots.append(mpf.make_addplot(df['RSI'], panel=2, color='purple', width=1))
            # RSI reference lines
            rsi_70 = pd.Series([70] * len(df), index=df.index)
            rsi_30 = pd.Series([30] * len(df), index=df.index)
            addplots.append(mpf.make_addplot(rsi_70, panel=2, color='red', linestyle='--', alpha=0.5))
            addplots.append(mpf.make_addplot(rsi_30, panel=2, color='green', linestyle='--', alpha=0.5))

        # Create the plot
        fig, axes = mpf.plot(
            df.set_index('DateTime'),
            type='candle',
            style='charles',
            volume=False,
            figsize=(16, 10),
            returnfig=True,
            addplot=addplots if addplots else None,
            title='Enhanced MACD Trading Strategy',
            panel_ratios=(3, 1, 1) if 'RSI' in df.columns else (3, 1),
            tight_layout=True
        )

        main_ax = axes[0]

        # Add signal markers
        if show_signals and 'Filtered_Crossover' in df.columns:
            self._add_signal_markers(main_ax, df)

        # Add SL/TP boxes
        if show_boxes and all(col in df.columns for col in ['SL', 'TP', 'Filtered_Crossover']):
            self._add_sl_tp_boxes(main_ax, df)

        return fig, axes

    def _add_signal_markers(self, ax, df: pd.DataFrame):
        """Add buy/sell signal markers to the chart."""
        for i, row in df.iterrows():
            if row['Filtered_Crossover'] == 1:  # Bullish
                ax.scatter(i, row['Low'] * 0.999, marker='^', color='green', s=100, zorder=5)
            elif row['Filtered_Crossover'] == -1:  # Bearish
                ax.scatter(i, row['High'] * 1.001, marker='v', color='red', s=100, zorder=5)

    def _add_sl_tp_boxes(self, ax, df: pd.DataFrame):
        """Add stop loss and take profit boxes to the chart."""
        for i, row in df.iterrows():
            if row['Filtered_Crossover'] != 0 and not pd.isna(row['SL']) and not pd.isna(row['TP']):
                width = 0.8
                color = 'green' if row['Filtered_Crossover'] == 1 else 'red'

                rect = patches.Rectangle(
                    (i - width/2, min(row['SL'], row['TP'])),
                    width,
                    abs(row['TP'] - row['SL']),
                    color=color,
                    alpha=0.2,
                    linewidth=1,
                    edgecolor=color
                )
                ax.add_patch(rect)

    def run_complete_analysis(self, csv_file: str) -> Tuple[pd.DataFrame, dict]:
        """
        Run the complete trading strategy analysis.

        Args:
            csv_file: Path to the CSV file

        Returns:
            Tuple of (processed DataFrame, backtest results)
        """
        print("Loading and preparing data...")
        df = self.load_and_prepare_data(csv_file)

        print("Calculating MACD indicators...")
        df = self.calculate_macd(df)

        print("Detecting crossovers...")
        df['Crossover'] = self.detect_macd_crossovers(df)

        print("Adding additional indicators...")
        df = self.add_additional_indicators(df)

        print("Calculating stop loss and take profit levels...")
        df = self.calculate_stop_loss_take_profit(df)

        print("Filtering signals...")
        df = self.filter_signals(df)

        print("Running backtest...")
        backtest_results = self.backtest_strategy(df)

        print("\n=== BACKTEST RESULTS ===")
        print(f"Total Trades: {backtest_results['total_trades']}")
        print(f"Win Rate: {backtest_results['win_rate']:.2f}%")
        print(f"Total P&L: ${backtest_results['total_pnl']:.2f}")
        print(f"Final Balance: ${backtest_results['final_balance']:.2f}")
        print(f"Max Drawdown: {backtest_results['max_drawdown']:.2f}%")

        return df, backtest_results


# Example usage
if __name__ == "__main__":
    # Initialize the strategy
    strategy = MACDTradingStrategy()

    # Run complete analysis
    try:
        df, results = strategy.run_complete_analysis("xauusdm15.filtered.csv")

        # Save results
        df.to_csv("enhanced_macd_analysis.csv")

        # Create enhanced chart for recent data
        fig, axes = strategy.plot_enhanced_chart(df, start_idx=23300)

        # Show the plot
        mpf.show()

    except FileNotFoundError:
        print("CSV file not found. Please ensure 'xauusdm15.filtered.csv' exists in the current directory.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
# Simple usage
# strategy = MACDTradingStrategy()
# df, results = strategy.run_complete_analysis("xauusdm15.filtered.csv")

# Customized parameters
strategy = MACDTradingStrategy(fast_period=8, slow_period=21, signal_period=5)
df, results = strategy.run_complete_analysis("xauusdm15.filtered.csv")
